In [78]:
%matplotlib inline
import pywt
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns


from pandas import set_option
set_option("display.max_rows", 10)
pd.options.mode.chained_assignment = None

filename = 'data/facies_vectors.csv'
training_data = pd.read_csv(filename)
training_data

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
0,3,A1 SH,SHRIMPLIN,2793.0,77.450,0.664,9.900,11.915,4.600,1,1.000
1,3,A1 SH,SHRIMPLIN,2793.5,78.260,0.661,14.200,12.565,4.100,1,0.979
2,3,A1 SH,SHRIMPLIN,2794.0,79.050,0.658,14.800,13.050,3.600,1,0.957
3,3,A1 SH,SHRIMPLIN,2794.5,86.100,0.655,13.900,13.115,3.500,1,0.936
4,3,A1 SH,SHRIMPLIN,2795.0,74.580,0.647,13.500,13.300,3.400,1,0.915
...,...,...,...,...,...,...,...,...,...,...,...
4144,5,C LM,CHURCHMAN BIBLE,3120.5,46.719,0.947,1.828,7.254,3.617,2,0.685
4145,5,C LM,CHURCHMAN BIBLE,3121.0,44.563,0.953,2.241,8.013,3.344,2,0.677
4146,5,C LM,CHURCHMAN BIBLE,3121.5,49.719,0.964,2.925,8.013,3.190,2,0.669
4147,5,C LM,CHURCHMAN BIBLE,3122.0,51.469,0.965,3.083,7.708,3.152,2,0.661


In [79]:
db3= pywt.Wavelet('db3')


In [80]:
grouped = training_data.groupby(['Well Name'])
max_GR_levels = []
max_ILD_levels = []
max_DeltaPHI_levels = []
max_PHIND_levels = []
max_PE_levels = []


for key in grouped.groups.keys():
    max_GR_levels.append(pywt.dwt_max_level(len(grouped.get_group(key).GR),db3))
    max_ILD_levels.append(pywt.dwt_max_level(len(grouped.get_group(key).ILD_log10),db3))
    max_DeltaPHI_levels.append(pywt.dwt_max_level(len(grouped.get_group(key).DeltaPHI),db3))
    max_PHIND_levels.append(pywt.dwt_max_level(len(grouped.get_group(key).PHIND),db3))
    max_PE_levels.append(pywt.dwt_max_level(len(grouped.get_group(key).PE),db3))

print(max_GR_levels,max_ILD_levels,max_DeltaPHI_levels,max_PHIND_levels,max_PE_levels)

[6, 6, 6, 4, 6, 6, 6, 6, 6, 6] [6, 6, 6, 4, 6, 6, 6, 6, 6, 6] [6, 6, 6, 4, 6, 6, 6, 6, 6, 6] [6, 6, 6, 4, 6, 6, 6, 6, 6, 6] [6, 6, 6, 4, 6, 6, 6, 6, 6, 6]


In [81]:
from scipy.interpolate import interp1d

In [82]:
def restore_df(group):
    return group.reset_index(drop=True)

In [83]:
n_levels = 6
new_vars = pd.DataFrame()

for key in grouped.groups.keys():
    
    depth = grouped.get_group(key)['Depth']
    temp_df = pd.DataFrame()
    temp_df['Depth'] = depth
    for log in ['GR']:
      
        temp_data = grouped.get_group(key)[log]
        
        for i in range(1,n_levels+1):

            cA,cD = pywt.dwt(temp_data,db3,mode='symmetric')

            new_depth = np.linspace(min(depth),max(depth),len(cD))
            f1 = interp1d(new_depth,cD,kind='linear')
            f2 = interp1d(new_depth,cD,kind='nearest')
            f3 = interp1d(new_depth,cA,kind='linear')
            
            if i==4:
#             temp_df[log + '_cD_lin_level_' + str(i)] = f1(depth)
                temp_df[log + '_cD_step_level_' + str(i)] = f2(depth)
#             temp_df[log + '_cA_level_' + str(i)] = f3(depth)
    
    temp_df['Well Name'] = [key for _ in range(len(depth))]
    new_vars = new_vars.append(temp_df)
                

In [84]:
new_vars = new_vars.sort_index()

In [85]:
new_vars.to_pickle('data/vars_from_dwt.pkl')

### Plot the result